In [1]:
import os
import sys
HOME_DIR = '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering'
os.chdir(HOME_DIR)

DATASETS_DIR = f'{HOME_DIR}/datasets'
DATA_DIR = f'{DATASETS_DIR}/11'

TRAIN_CSV_FILE = f'{DATA_DIR}/bounding_boxes/train_labels.csv'
TRAIN_IMAGES_DIR = f'{DATA_DIR}/images/train'
TRAIN_LABELS_DIR = f'{DATA_DIR}/labels/train'
SELECTED_TRAIN_IMAGES_DIR= f'{DATA_DIR}/images/train_selected'
SELECTED_TRAIN_LABELS_DIR = f'{DATA_DIR}/labels/train_selected'


TEST_CSV_FILE = f'{DATA_DIR}/bounding_boxes/test_labels.csv'
TEST_IMAGES_DIR = f'{DATA_DIR}/images/test'
TEST_LABELS_DIR = f'{DATA_DIR}/labels/test'
SELECTED_TEST_IMAGES_DIR = f'{DATA_DIR}/images/test_selected'
SELECTED_TEST_LABELS_DIR = f'{DATA_DIR}/labels/test_selected'

OUTPUT_IMAGES_DIR = 'week-06-portfolio/evaluation_images'

yaml_file_path = f'{DATA_DIR}/graffiti.yaml'

print(f'HOME_DIR: {HOME_DIR}')
print(f'DATASETS_DIR: {DATASETS_DIR}')
print(f'DATA_DIR: {DATA_DIR}')
print(f'TRAIN_CSV_FILE: {TRAIN_CSV_FILE}')
print(f'TRAIN_IMAGES_DIR: {TRAIN_IMAGES_DIR}')
print(f'TRAIN_LABELS_DIR: {TRAIN_LABELS_DIR}')
print(f'TEST_CSV_FILE: {TEST_CSV_FILE}')
print(f'TEST_IMAGES_DIR: {TEST_IMAGES_DIR}')
print(f'TEST_LABELS_DIR: {TEST_LABELS_DIR}')

HOME_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering
DATASETS_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets
DATA_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11
TRAIN_CSV_FILE: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/bounding_boxes/train_labels.csv
TRAIN_IMAGES_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/train
TRAIN_LABELS_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/labels/train
TEST_CSV_FILE: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/bounding_boxes/test_labels.csv
TEST_IMAGES_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test
TEST_LABELS_DIR: /Users/anhdang/Documents/Github/COS

In [2]:
!pip install --upgrade pip --quiet
!pip install -r requirements.txt --quiet

In [12]:
from ultralytics import YOLO
import platform
import torch
import pandas as pd
import os
from tqdm import tqdm
import torch
from torchvision.ops import box_iou
import pandas as pd
from PIL import Image
from tqdm import tqdm
import random
import shutil
import cv2
import torch
from torchvision.ops import box_iou
import pandas as pd
from PIL import Image
from tqdm import tqdm


In [3]:
if platform.system() == 'Darwin':
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: mps


In [4]:
def convert_annotations(csv_file, images_dir, output_dir, class_mapping):
    df = pd.read_csv(csv_file)
    grouped = df.groupby('filename')

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename, group in tqdm(grouped, desc=f'Converting annotations for {csv_file}'):
        image_path = os.path.join(images_dir, filename)
        if not os.path.exists(image_path):
            continue  # Skip if image does not exist

        img_width = group.iloc[0]['width']
        img_height = group.iloc[0]['height']

        annotations = []
        for _, row in group.iterrows():
            class_id = class_mapping[row['class']]
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']

            # Convert to YOLO format
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            bbox_width = (xmax - xmin) / img_width
            bbox_height = (ymax - ymin) / img_height

            annotations.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

        # Write annotations to file
        txt_filename = os.path.splitext(filename)[0] + '.txt'
        with open(os.path.join(output_dir, txt_filename), 'w') as f:
            for ann in annotations:
                f.write(ann + '\n')



In [5]:
# Define class mapping
class_mapping = {'Graffiti': 0}

# Convert training annotations
convert_annotations(TRAIN_CSV_FILE, TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR, class_mapping)

# Convert test annotations
convert_annotations(TEST_CSV_FILE, TEST_IMAGES_DIR, TEST_LABELS_DIR, class_mapping)


Converting annotations for /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/bounding_boxes/train_labels.csv: 100%|██████████| 813/813 [00:00<00:00, 4154.74it/s]
Converting annotations for /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/bounding_boxes/test_labels.csv: 100%|██████████| 209/209 [00:00<00:00, 3523.56it/s]


In [6]:
def select_random_images(source_dir, dest_dir, num_images, used_images=set()):
    images = [f for f in os.listdir(source_dir) if f.endswith('.jpg') and f not in used_images]
    selected_images = random.sample(images, min(num_images, len(images)))
    used_images.update(selected_images)

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for img in selected_images:
        shutil.copy(os.path.join(source_dir, img), os.path.join(dest_dir, img))
    return used_images

def copy_annotation_files(image_dir, label_source_dir, label_dest_dir):
    if not os.path.exists(label_dest_dir):
        os.makedirs(label_dest_dir)
    for img_file in os.listdir(image_dir):
        if img_file.endswith('.jpg'):
            base_name = os.path.splitext(img_file)[0]
            label_file = base_name + '.txt'
            src_label_path = os.path.join(label_source_dir, label_file)
            dst_label_path = os.path.join(label_dest_dir, label_file)
            if os.path.exists(src_label_path):
                shutil.copy(src_label_path, dst_label_path)




In [7]:
# Set random seed for reproducibility
random.seed(42)

# Initialize used images sets
used_train_images = set()
used_test_images = set()

# Select 400 random training images
used_train_images = select_random_images(TRAIN_IMAGES_DIR, SELECTED_TRAIN_IMAGES_DIR, 400, used_train_images)

# Copy corresponding training annotation files
copy_annotation_files(SELECTED_TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR, SELECTED_TRAIN_LABELS_DIR)

# Select 40 random test images
used_test_images = select_random_images(TEST_IMAGES_DIR, SELECTED_TEST_IMAGES_DIR, 40, used_test_images)

# Copy corresponding test annotation files
copy_annotation_files(SELECTED_TEST_IMAGES_DIR, TEST_LABELS_DIR, SELECTED_TEST_LABELS_DIR)

In [8]:
def create_yaml_file(file_path, train_images, val_images, nc, class_names):
    data = {
        'train': train_images,
        'val': val_images,
        'nc': nc,
        'names': class_names
    }
    with open(file_path, 'w') as f:
        for key, value in data.items():
            if isinstance(value, list):
                value_str = str(value).replace("'", "")
                f.write(f"{key}: {value_str}\n")
            else:
                f.write(f"{key}: {value}\n")

In [9]:
# Usage
train_images_path = os.path.abspath(SELECTED_TRAIN_IMAGES_DIR)
val_images_path = os.path.abspath(SELECTED_TEST_IMAGES_DIR)  # Using test data as validation set

nc = 1
class_names = ['Graffiti']

create_yaml_file(yaml_file_path, train_images_path, val_images_path, nc, class_names)

In [10]:
from ultralytics import YOLO

# Load a pretrained YOLOv5s model
model = YOLO('week-06-portfolio/models/yolov5su.pt')  # Ensure yolov5s.pt is in your current directory or provide the correct path

# Train the model
results = model.train(data=yaml_file_path, epochs=10, imgsz=640, batch=16, name='graffiti_detection', device=device)


engine/trainer: task=detect, mode=train, model=week-06-portfolio/models/yolov5su.pt, data=/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/graffiti.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=graffiti_detection7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/labels/train_selected... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<00:00, 2470.05it/s]

train: New cache created: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/labels/train_selected.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/labels/test_selected... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<00:00, 2944.46it/s]

...

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:12<00:12, 12.79s/it]

WARNING ⚠️ NMS time limit 2.400s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:18<00:00,  9.37s/it]


                   all         40         80      0.408      0.212      0.168      0.104
Speed: 3.7ms preprocess, 87.9ms inference, 0.0ms loss, 158.3ms postprocess per image
Results saved to /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/runs/detect/graffiti_detection7


In [10]:
def compute_iou(pred_boxes, true_boxes):
    # Convert boxes to tensors
    pred_boxes = torch.tensor(pred_boxes)
    true_boxes = torch.tensor(true_boxes)

    # Compute IoU
    iou = box_iou(pred_boxes, true_boxes)
    return iou.diag().numpy()  # Get IoUs for matched boxes

def evaluate_model(model, images_dir, labels_dir, output_images_dir=None):
    results = []
    images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

    if output_images_dir and not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)

    for img_name in tqdm(images, desc='Evaluating model'):
        img_path = os.path.join(images_dir, img_name)
        label_path = os.path.join(labels_dir, os.path.splitext(img_name)[0] + '.txt')

        # Perform inference
        preds = model.predict(img_path, conf=0.25)

        # Load image for drawing
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[:2]

        # Get predicted boxes and confidence scores
        pred_boxes = []
        confidences = []
        for pred in preds:
            for box in pred.boxes:
                x_min = box.xyxy[0][0].item()
                y_min = box.xyxy[0][1].item()
                x_max = box.xyxy[0][2].item()
                y_max = box.xyxy[0][3].item()
                conf = box.conf.item()
                pred_boxes.append([x_min, y_min, x_max, y_max])
                confidences.append(conf)

                # Draw predicted bounding box
                if output_images_dir:
                    label = f"{model.names[int(box.cls)]}: {conf:.2f}"
                    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
                    cv2.putText(img, label, (int(x_min), int(y_min) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Get true boxes
        true_boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    # Convert back to absolute coordinates
                    x_center *= img_width
                    y_center *= img_height
                    width *= img_width
                    height *= img_height
                    x_min = x_center - width / 2
                    y_min = y_center - height / 2
                    x_max = x_center + width / 2
                    y_max = y_center + height / 2
                    true_boxes.append([x_min, y_min, x_max, y_max])

                    # Draw true bounding box
                    if output_images_dir:
                        cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 255), 2)

        # Save image with drawn bounding boxes
        if output_images_dir:
            output_image_path = os.path.join(output_images_dir, img_name)
            cv2.imwrite(output_image_path, img)

        # Compute IoU
        if pred_boxes and true_boxes:
            ious = compute_iou(pred_boxes, true_boxes)
            max_iou = max(ious)
            max_conf = confidences[ious.argmax()]
        else:
            max_iou = 0.0
            max_conf = 0.0 if not confidences else max(confidences)

        results.append({
            'image_name': img_name,
            'confidence_value': max_conf,
            'IoU_value': max_iou
        })

    return pd.DataFrame(results)



In [12]:
# Usage
df_results = evaluate_model(model, SELECTED_TEST_IMAGES_DIR, SELECTED_TEST_LABELS_DIR, OUTPUT_IMAGES_DIR)
df_results.to_csv('week-06-portfolio/evaluation_results.csv', index=False)

Evaluating model:   0%|          | 0/40 [00:00<?, ?it/s]


image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180713_105001.jpg: 640x480 13 Graffitis, 4247.7ms
Speed: 141.5ms preprocess, 4247.7ms inference, 2023.8ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:   2%|▎         | 1/40 [00:07<04:48,  7.40s/it]


image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20100101_032102.jpg: 640x480 14 Graffitis, 68.9ms
Speed: 4.8ms preprocess, 68.9ms inference, 1592.2ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:   5%|▌         | 2/40 [00:09<02:37,  4.13s/it]

...

Evaluating model:  90%|█████████ | 36/40 [00:22<00:01,  2.77it/s]


image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180725_122231.jpg: 640x480 8 Graffitis, 28.1ms
Speed: 2.6ms preprocess, 28.1ms inference, 12.9ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180720_142111.jpg: 640x480 17 Graffitis, 21.2ms
Speed: 2.6ms preprocess, 21.2ms inference, 399.5ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:  95%|█████████▌| 38/40 [00:23<00:00,  3.10it/s]


image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180724_121144.jpg: 640x480 7 Graffitis, 27.3ms
Speed: 1.9ms preprocess, 27.3ms inference, 8.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180714_102857.jpg: 640x480 5 Graffitis, 27.6ms
Speed: 1.7ms preprocess, 27.6ms inference, 34.8ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model: 100%|██████████| 40/40 [00:23<00:00,  1.71it/s]


In [16]:
# Initialize variables for iterative training
iou_threshold = 0.9
satisfied = False
iteration = 1

while not satisfied:
    print(f"\nStarting iteration {iteration}")

    # Select new training images
    training_images = random.sample(os.listdir(TRAIN_IMAGES_DIR), 400)

    # Update training data directories
    selected_train_images_dir = f'week-06-portfolio/images/train_selected_iter_{iteration}'
    selected_train_labels_dir = f'week-06-portfolio/labels/train_selected_iter_{iteration}'
    if not os.path.exists(selected_train_images_dir):
        os.makedirs(selected_train_images_dir)
    if not os.path.exists(selected_train_labels_dir):
        os.makedirs(selected_train_labels_dir)

    # Copy selected images and annotations
    for img in training_images:
        shutil.copy(os.path.join(TRAIN_IMAGES_DIR, img), os.path.join(selected_train_images_dir, img))
        label_file = os.path.splitext(img)[0] + '.txt'
        src_label_path = os.path.join(TRAIN_LABELS_DIR, label_file)
        dst_label_path = os.path.join(selected_train_labels_dir, label_file)
        if os.path.exists(src_label_path):
            shutil.copy(src_label_path, dst_label_path)

    # Update YAML file
    train_images_path = os.path.abspath(selected_train_images_dir)
    val_images_path = os.path.abspath(SELECTED_TEST_IMAGES_DIR)

    # Update the YAML file path for this iteration
    yaml_file_path_iter = f'week-06-portfolio/yaml/graffiti_iter_{iteration}.yaml'
    create_yaml_file(yaml_file_path_iter, train_images_path, val_images_path, nc, class_names)

    # Load the model from previous iteration
    if iteration == 1:
        model = YOLO('week-06-portfolio/models/yolov5su.pt')  # Start with pre-trained YOLOv5su model
    else:
        previous_model_path = f'week-06-portfolio/runs/train/graffiti_detection_iter_{iteration - 1}/weights/best.pt'
        model = YOLO(previous_model_path)

    # Train the model with 5 epochs
    model.train(
        data=yaml_file_path_iter,
        epochs=5,  # Lowered epochs to 5 as per your earlier request
        imgsz=640,
        batch=16,
        project='week-06-portfolio/runs/train',
        name=f'graffiti_detection_iter_{iteration}',
        device=device
    )

    # Evaluate the model
    output_images_dir_iter = f'week-06-portfolio/evaluation_images_iter_{iteration}'
    if not os.path.exists(output_images_dir_iter):
        os.makedirs(output_images_dir_iter)

    df_results = evaluate_model(model, SELECTED_TEST_IMAGES_DIR, SELECTED_TEST_LABELS_DIR, output_images_dir_iter)
    df_results.to_csv(f'week-06-portfolio/evaluation_results_iter_{iteration}.csv', index=False)

    # Check IoU threshold
    over_threshold = df_results[df_results['IoU_value'] > iou_threshold]
    if len(over_threshold) / len(df_results) >= 0.8:
        print(f"IoU threshold met in iteration {iteration}")
        satisfied = True
    else:
        print(f"IoU threshold not met in iteration {iteration}")

    # Prepare for next iteration
    iteration += 1


if not satisfied:
    print("IoU threshold not met after all iterations.")
else:
    print("Training completed successfully.")




Starting iteration 1
engine/trainer: task=detect, mode=train, model=week-06-portfolio/models/yolov5su.pt, data=week-06-portfolio/yaml/graffiti_iter_1.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=week-06-portfolio/runs/train, name=graffiti_detection_iter_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

train: Scanning /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/week-06-portfolio/labels/train_selected_iter_1... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<00:00, 3062.90it/s]

train: New cache created: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/week-06-portfolio/labels/train_selected_iter_1.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/labels/test_selected.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]

Plotting labels to week-06-portfolio/runs/train/graffiti_detection_iter_1/labels.jpg... 


...

WARNING ⚠️ NMS time limit 2.400s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:17<00:00,  8.50s/it]


                   all         40         80      0.439      0.113     0.0955     0.0653

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/5         0G      1.468      1.898      1.338         96        640:  56%|█████▌    | 14/25 [02:19<01:49,  9.96s/it]


KeyboardInterrupt: 

In [ ]:
model.export()